In [123]:
import pandas as pd
import numpy as np
import random as rnd

from datetime import datetime

import pickle
from tqdm.notebook import tqdm

In [124]:
import seaborn as sns
import matplotlib.pyplot as plt

In [125]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split

Укажите путь к директории с датафреймом `data.txt`

In [126]:
working_directory = 'drive/MyDrive/mindset_prolong'

In [175]:
ls -l $working_directory

ls: cannot access 'working_directory': No such file or directory


In [128]:
data = pd.read_csv('{}/{}'.format(working_directory, 'data.txt'), sep=';')
data

,DATA_TYPE,POLICY_ID,POLICY_BEGIN_MONTH,POLICY_END_MONTH,POLICY_IS_RENEWED,POLICY_SALES_CHANNEL,POLICY_SALES_CHANNEL_GROUP,POLICY_BRANCH,POLICY_MIN_AGE,POLICY_MIN_DRIVING_EXPERIENCE,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_ENGINE_POWER,VEHICLE_IN_CREDIT,VEHICLE_SUM_INSURED,POLICY_INTERMEDIARY,INSURER_GENDER,POLICY_CLM_N,POLICY_CLM_GLT_N,POLICY_PRV_CLM_N,POLICY_PRV_CLM_GLT_N,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_COURT_SIGN,CLAIM_AVG_ACC_ST_PRD,POLICY_HAS_COMPLAINTS,POLICY_YEARS_RENEWED_N,POLICY_DEDUCT_VALUE,CLIENT_REGISTRATION_REGION,POLICY_PRICE_CHANGE
0,TRAIN,1,1,1,1,39,1,Москва,51,12,Land Rover,Discovery,245.0,0,1283202.0,856,F,0,0,N,N,1,0,0,0.0,0,0,0.0,Тульская,-1.00
1,TRAIN,2,1,1,1,50,5,Москва,35,7,Opel,Zafira,140.0,0,453000.0,N,M,0,0,0,0,1,1,0,0.0,0,4,0.0,Москва,-0.05
2,TRAIN,3,1,1,1,52,6,Москва,41,6,Kia,Soul,129.0,0,647718.0,N,F,0,0,1L,1L,0,0,0,0.0,0,1,12518.0,Московская,-0.07
3,TRAIN,4,1,1,1,50,5,Москва,36,12,Citroen,C-Crosser,170.0,0,757795.0,N,M,0,0,0,0,1,1,0,0.0,0,6,15000.0,Москва,0.05
4,TRAIN,5,1,1,0,52,6,Санкт-Петербург,42,5,Renault,Logan,102.0,0,604500.0,171,F,0,0,N,N,0,0,0,0.0,0,0,50000.0,Ленинградская,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96600,TRAIN,96601,3,3,1,55,6,Москва,38,6,Ford,Mondeo,199.0,0,1799000.0,1096,M,0,0,N,N,1,1,0,0.0,0,0,0.0,Московская,-0.14
96601,TRAIN,96602,4,3,1,50,5,Санкт-Петербург,30,6,Mercedes-Benz,GL,249.0,0,3000500.0,N,M,0,0,0,0,1,1,0,0.0,0,1,0.0,Санкт-Петербург,0.00
96602,TRAIN,96603,3,3,1,50,5,Москва,59,31,Opel,Meriva,140.0,0,561700.0,N,M,1S,1S,0,0,1,1,0,3.0,0,2,0.0,Москва,-0.22
96603,TRAIN,96604,3,3,1,52,6,Москва,41,19,Toyota,Land Cruiser Prado,173.0,0,1783992.0,910,M,0,0,0,0,0,1,0,0.0,0,1,0.0,Московская,0.04


# Оценим параметры, отберём нужные и подготовим данные к обучению

Напишем функцию для оценки влияния параметра на пролонгацию полиса

In [129]:
def check_sum_count(column):
    result = data[['POLICY_IS_RENEWED', column]].groupby(column)['POLICY_IS_RENEWED'].agg(['count', 'sum'])
    result['result'] = result['sum'] / result['count']
    result = result.copy().sort_values(by='count', ascending=False)
    return result

По итогам проверки параметров датафрейма с помощью функции `check_sum_count` были выбраны следующие параметры:

In [130]:
no_changes_need_ = [
                    'DATA_TYPE',
                    'POLICY_ID',
                    'POLICY_IS_RENEWED',
                    'VEHICLE_IN_CREDIT',
                    'CLIENT_HAS_DAGO',
                    'CLIENT_HAS_OSAGO',
                    'POLICY_HAS_COMPLAINTS',
                    ]
dummies_ = {
                    'POLICY_CLM_GLT_N':     ['0', '1S', '2', '1L', '4+', '3', 'n/d'],
                    'POLICY_CLM_N':         ['0', '1S', '2', '1L', '3', '4+', 'n/d'],
                    'POLICY_PRV_CLM_GLT_N': ['N', '0', '1L', '1S', '2', '3', '4+'],
                    'POLICY_PRV_CLM_N':     ['N', '0', '1L', '1S', '2', '3', '4+']
                    }
true_false_ = {
                    'POLICY_BRANCH': lambda x: int(x == 'Москва'),
                    'INSURER_GENDER': lambda x: int(x == 'M'),
                    'CLAIM_AVG_ACC_ST_PRD': lambda x: int(int(x) == 0),
                    'POLICY_DEDUCT_VALUE': lambda x: int(int(x) == 0),
                    }
to_categories_ = {
                    'POLICY_MIN_AGE':                   [[18, 34], [35, 41], [42, 50], [51, float('inf')],],
                    'POLICY_MIN_DRIVING_EXPERIENCE':    [[0, 8], [9, 14], [15, 19], [20, float('inf')],],
                    'VEHICLE_ENGINE_POWER':             [[0., 117], [118, 140], [141, 146], [147, 150], [151, 180], [181, float('inf')],],
                    'VEHICLE_SUM_INSURED':              [[0., 550_000], [550_001, 810_000], [810_001, 1_160_000], [1_160_001, float('inf')],],
                    'POLICY_YEARS_RENEWED_N':           [[0, 0], [1, 1], [2, float('inf')]],
                    }

In [131]:
df = data[no_changes_need_].copy()

Приведём некоторые данные к числовому виду

пункт `POLICY_YEARS_RENEWED_N` содержит поле `N`, но так как количество таких значений невелико предположим что данное поле означает `0` и привидем значения к типу `int32`

In [132]:
check_sum_count('POLICY_YEARS_RENEWED_N')

,count,sum,result
POLICY_YEARS_RENEWED_N,,,
0,33249,14142,0.425336
1,25299,13667,0.540219
2,15278,7928,0.518916
3,11496,6310,0.548887
4,6215,3342,0.537731
5,2228,1319,0.592011
6,1748,1021,0.584096
7,832,481,0.578125
8,104,69,0.663462


In [133]:
data['POLICY_YEARS_RENEWED_N'] = data['POLICY_YEARS_RENEWED_N'].replace('N', '0')
data['POLICY_YEARS_RENEWED_N'] = data['POLICY_YEARS_RENEWED_N'].astype('int32')

пункт `POLICY_MIN_DRIVING_EXPERIENCE` содержит данные в формате `год начала вождения` вместо `лет стажа` исправим это предполагая что таблица сформирована в 2021 году

In [134]:
def fix_driving_experience(x):
    if x > 1000:
        return datetime.now().year - x
    else:
        return x

data['POLICY_MIN_DRIVING_EXPERIENCE'] = data['POLICY_MIN_DRIVING_EXPERIENCE'].apply(fix_driving_experience)

Переведем параметры из списка `dummies_` в бинарные с помощью функции `get_dummies`

In [135]:
dummies_columns_test = list()

for column, list_dummies in dummies_.items():
    for dummy in list_dummies:
        dummies_columns_test.append('{}_{}'.format(column, dummy))
        df['{}_{}'.format(column, dummy)] = data[column].apply(lambda x: int(x == dummy))


Переведем параметры из списка `true_false_` в бинарные с помощью заранее определённых `lambda`-функций

In [136]:
for column, function in true_false_.items():
    df[column] = data[column].apply(function)

Переведем параметры из списка `to_categories_` в бинарные, разбив на категории

In [137]:
for column, categories in to_categories_.items():
    for category in categories:
        df['{}_{}_{}'.format(column, category[0], category[1])] = data[column].apply(lambda x: int(category[0] <= x and x <= category[1]))

# Обучение

In [138]:
train = df[df['DATA_TYPE'] == 'TRAIN'].drop(columns=['DATA_TYPE', 'POLICY_ID'])
test = df[df['DATA_TYPE'] != 'TRAIN'].drop(columns=['DATA_TYPE', 'POLICY_ID', 'POLICY_IS_RENEWED'])



In [139]:
# test_json = data[data['DATA_TYPE'] != 'TRAIN'].drop(columns='DATA_TYPE').copy()
# test_json

In [140]:
y = train['POLICY_IS_RENEWED']
X = train.drop(columns=['POLICY_IS_RENEWED'])

In [141]:
X_train, X_val, y_train, y_val = train_test_split(X, y)

In [142]:
model = RandomForestClassifier(criterion='gini', min_samples_leaf=6, min_samples_split=6, n_estimators=500)

In [143]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=6,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [144]:
predictions = model.predict(X_val)

In [145]:
accuracy_score(y_val, predictions)

0.6600868127325341

In [146]:
ids

9           10
11          12
12          13
21          22
36          37
         ...  
96573    96574
96576    96577
96585    96586
96594    96595
96597    96598
Name: POLICY_ID, Length: 19198, dtype: int64

In [147]:
test

,VEHICLE_IN_CREDIT,CLIENT_HAS_DAGO,CLIENT_HAS_OSAGO,POLICY_HAS_COMPLAINTS,POLICY_CLM_GLT_N_0,POLICY_CLM_GLT_N_1S,POLICY_CLM_GLT_N_2,POLICY_CLM_GLT_N_1L,POLICY_CLM_GLT_N_4+,POLICY_CLM_GLT_N_3,POLICY_CLM_GLT_N_n/d,POLICY_CLM_N_0,POLICY_CLM_N_1S,POLICY_CLM_N_2,POLICY_CLM_N_1L,POLICY_CLM_N_3,POLICY_CLM_N_4+,POLICY_CLM_N_n/d,POLICY_PRV_CLM_GLT_N_N,POLICY_PRV_CLM_GLT_N_0,POLICY_PRV_CLM_GLT_N_1L,POLICY_PRV_CLM_GLT_N_1S,POLICY_PRV_CLM_GLT_N_2,POLICY_PRV_CLM_GLT_N_3,POLICY_PRV_CLM_GLT_N_4+,POLICY_PRV_CLM_N_N,POLICY_PRV_CLM_N_0,POLICY_PRV_CLM_N_1L,POLICY_PRV_CLM_N_1S,POLICY_PRV_CLM_N_2,POLICY_PRV_CLM_N_3,POLICY_PRV_CLM_N_4+,POLICY_BRANCH,INSURER_GENDER,CLAIM_AVG_ACC_ST_PRD,POLICY_DEDUCT_VALUE,POLICY_MIN_AGE_18_34,POLICY_MIN_AGE_35_41,POLICY_MIN_AGE_42_50,POLICY_MIN_AGE_51_inf,POLICY_MIN_DRIVING_EXPERIENCE_0_8,POLICY_MIN_DRIVING_EXPERIENCE_9_14,POLICY_MIN_DRIVING_EXPERIENCE_15_19,POLICY_MIN_DRIVING_EXPERIENCE_20_inf,VEHICLE_ENGINE_POWER_0.0_117,VEHICLE_ENGINE_POWER_118_140,VEHICLE_ENGINE_POWER_141_146,VEHICLE_ENGINE_POWER_147_150,VEHICLE_ENGINE_POWER_151_180,VEHICLE_ENGINE_POWER_181_inf,VEHICLE_SUM_INSURED_0.0_550000,VEHICLE_SUM_INSURED_550001_810000,VEHICLE_SUM_INSURED_810001_1160000,VEHICLE_SUM_INSURED_1160001_inf,POLICY_YEARS_RENEWED_N_0_0,POLICY_YEARS_RENEWED_N_1_1,POLICY_YEARS_RENEWED_N_2_inf
9,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0
11,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0
12,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0
21,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1
36,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96573,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
96576,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0
96585,1,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1
96594,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0


In [148]:
ids = df[df['DATA_TYPE'] != 'TRAIN']['POLICY_ID'].copy()

result = pd.DataFrame({'POLICY_ID': ids, 'POLICY_IS_RENEWED': model.predict(test), 'POLICY_IS_RENEWED_PROBABILITY': 0})
result.to_csv('{}/predictions_{}.csv'.format(working_directory, datetime.now().strftime('%m%d%Y_%H_%M_%S')), index=False)

# Подбор гиперпараметров

In [149]:
# model_precision = precision_score(y_val, predictions)
# model_recall = recall_score(y_val, predictions)
# model_f1_score = f1_score(y_val, predictions)
# metrics = pd.DataFrame(['model_precision', 'model_recall', 'model_f1_score'], [model_precision, model_recall, model_f1_score])

In [150]:
# metrics

In [151]:
# models = {
#     'Logistic Regression': LogisticRegression(),
#     'Naive Bayes': MultinomialNB(),
#     'Decision Tree': DecisionTreeClassifier(),
#     'Support Vector Machines': LinearSVC(),
#     'Random Forest': RandomForestClassifier(),
# }

In [152]:
# metrics = pd.DataFrame([])

# for model_name in tqdm(models.keys()):
#     model = models[model_name]
#     model.fit(X_train, y_train)
#     predictions = model.predict(X_val)
#     model_accuracy = accuracy_score(y_val, predictions)
#     metrics[model_name] = [model_accuracy]

In [153]:
# metrics

In [154]:
# predictions[predictions == y_val]

In [155]:
def metrics_calculate(validation_v, predictions_v):
    val_negative = y_val.size - sum(y_val)
    val_positive = y_val.size - val_negative
    pred = predictions_v.copy()
    val = validation_v.copy()

    result = val[val == pred]
    res_negative = result.size - sum(result)
    res_positive = result.size - res_negative

    del result
    del pred
    del val
    return res_positive / val_positive, res_negative / val_negative


In [156]:
# metrics_calculate(y_val, predictions)

In [157]:
# accuracy_score(y_val, predictions)



---



In [158]:
# param_grid = {
#     'Logistic Regression': {
#         'penalty' : ['l1', 'l2'],
#         'solver' : ['liblinear', 'saga'],
#         'max_iter': [100, 300, 500, 700, 1000]
#     },
#     'Naive Bayes': {
#         'fit_prior': [False, True]
#     },
#     'Decision Tree': {
#         'criterion': ['gini', 'entropy'],
#         'max_depth': [None, 50, 100, 150],
#         'min_samples_split': np.linspace(2, 5, 4, dtype='int'),
#         'min_samples_leaf': np.linspace(1, 3, 3, dtype='int')
#     },
#     'Support Vector Machines': {
#         'loss': ['hinge', 'squared_hinge'],
#         'max_iter': [1000, 2000],
#     },
#     'Random Forest':
#     {
#         'n_estimators': [300, 400, 500],
#         'criterion': ['gini', 'entropy'],
#         'max_depth': [None],
#         'min_samples_split': [4, 6],
#         'min_samples_leaf': [4, 6]
#     }
# }

In [159]:
# best_params = dict()

# for model_name in tqdm(models.keys()):
#     model = models[model_name]
#     model_grid = param_grid[model_name]
#     gs = GridSearchCV(model, model_grid, return_train_score=True, n_jobs=-1, verbose=True)
#     gs.fit(X_train, y_train)
#     best_params[model_name] = gs.best_params_

In [160]:
# model = RandomForestClassifier()
# model_grid = param_grid['Random Forest']
# gs = GridSearchCV(model, model_grid, return_train_score=True, n_jobs=-1, verbose=True)
# gs.fit(X_train, y_train)
# best_params['Random Forest'] = gs.best_params_

In [161]:
# best_params

In [162]:
# {'Random Forest': {'criterion': 'gini',
#   'max_depth': None,
#   'min_samples_leaf': 6,
#   'min_samples_split': 6,
#   'n_estimators': 500}}

In [163]:
# model_rf = RandomForestClassifier(criterion='gini', min_samples_leaf=6, min_samples_split=6, n_estimators=500)

In [164]:
# model_rf.fit(X_train, y_train)

In [165]:
# predictions_rf = model_rf.predict(X_val)

In [166]:
# metrics_calculate(y_val, predictions_rf)

In [167]:
# accuracy_score(y_val, predictions_rf)

# Выгрузка модели для передачи её приложению server.py

Сохраним обученную модель в формате `pickle`

In [168]:
pickle.dump(model, open('{}/{}'.format(working_directory, 'model.pkl'),'wb'))

Теперь данную модель необходимо переместить в папку `01_server`  и можно будет запустить сервис в `docker`



---



Для тестирования приложения `client.py` необходимо создать срез датафрейма в формате `.csv`

In [169]:
# создадим таблицу с данными для отправки на сервер
# csv_predict = data[data['DATA_TYPE'] != 'TRAIN'].drop(columns='DATA_TYPE').copy()
# csv_predict.to_csv('{}/data_to_predict.csv'.format(working_directory), index=False)